In [ ]:
# !pip3 install pendulum
!pip3 install impyla

In [22]:
# -*- coding=utf-8 -*-

import os
import pandas as pd
import pendulum 
from impala.dbapi import connect


os.environ.update(
    {
        "HIVESERVER2_API_KEY": "9954b1025d9abd6f31b4a3cb6b190a9830Uop36Pa5m",
        "HIVESERVER2_API_SECRET": "UPM9rnL6tJ9G80l6Ci6ZcjdS55ZizCHTW3G0Wvx5Ks",
    }
)

DEFAULT_API_KEY = os.environ.get('HIVESERVER2_API_KEY')
DEFAULT_API_SECRET = os.environ.get('HIVESERVER2_API_SECRET')
DEFAULT_HOST = os.environ.get('HIVESERVER2_HOST') or 'bbx.lilithgame.com'
DEFAULT_PORT = os.environ.get('HIVESERVER2_PORT') or 10000


class SQLClient:
    _engine_types = set(["trino", "hive", "impala"])
    
    def __init__(self, username, host=DEFAULT_HOST, port=DEFAULT_PORT, 
                 api_key=DEFAULT_API_KEY, api_secret=DEFAULT_API_SECRET, **kwargs):
        self._username = username
        self._host = host
        self._port = port
        self._api_key = api_key
        self._api_secret = api_secret
        self._timeout = kwargs.get("timeout", 600)
        self._conn = None
        self._cursor = None
    
    def reset(self):
        config = {
            'host': self._host,
            'port': self._port,
            'user': self._api_key,
            'password': self._api_secret,
            "auth_mechanism": 'plain',
            "timeout": self._timeout,
        }
        self._conn = connect(**config)
        self._cursor = None

    @property
    def connection(self):
        if not self._conn:
            self.reset()
        return self._conn
    
    @property
    def cursor(self):
        if not self._cursor:
            self._cursor = self.connection.cursor()
        return self._cursor
    
    @property
    def description(self):
        return self._cursor.description if self._cursor else None
    
    def close(self):
        if self._conn:
            self._conn.close()

    def close_cursor(self):
        if self._cursor:
            try:
                self._cursor.close()
            except Exception as e:
                self.reset()
                print(e)
            self._cursor = None
    
    def fetchone(self):
        return self._cursor.fetchone()

    def fetchmany(self, size=None):
        return self._cursor.fetchmany(size)
    
    def fetchall(self):
        res = self._cursor.fetchall()
        self.close_cursor()
        return res
    
    def to_pandas(self):
        col = [r[0] for r in self._cursor.description]
        df = pd.DataFrame(self._cursor.fetchall(), columns=col)
        self.close_cursor()
        return df
        
    def execute_query(self, sql, engine_type, query_queue=None, username=None, **kwargs):
        self.close_cursor()
        
        assert engine_type in SQLClient._engine_types
        
        self._cursor = cursor = self.cursor
        
        username = username or self._username
        
        if username is not None:
            cursor.execute(f"set lilith.dispatcher.exec.engine.query.operator={username}")
        if query_queue is not None:
            cursor.execute(f"set lilith.dispatcher.exec.engine.query.queue.={query_queue}")
        if engine_type is not None:
            cursor.execute(f"set lilith.dispatcher.exec.engine.type={engine_type}")
        cursor.execute(sql)
        
    def execute_hive_query(self, sql, query_queue=None, username=None, **kwargs):
        self.execute_query(sql, engine_type="hive", query_queue=query_queue, username=username, **kwargs)
    
    def execute_impala_query(self, sql, query_queue=None, username=None, **kwargs):
        self.execute_query(sql, engine_type="impala", query_queue=query_queue, username=username, **kwargs)
    
    def execute_trino_query(self, sql, query_queue=None, username=None, **kwargs):
        self.execute_query(sql, engine_type="trino", query_queue=query_queue, username=username, **kwargs)    


In [50]:

from tqdm import trange
from jinja2 import Template


client = SQLClient('mixx')

DS = pendulum.Date(2024,5,31)

fea_list=['acc_first_purchase_price_usd',
 'acc_role_cnt',
 'acc_total_charge_cnt',
 'bundle_pop_buy_max_price_tech_unlock_trigger',
 'purchase_price_max_last_3days',
 'purchase_price_1499_last_week',
 'purchase_price_1499_last_week_adjust',
 'purchase_price_1999_last_week_adjust',
 'purchase_price_2999_last_week_adjust',
 'purchase_price_4999_last_week_adjust',
 'purchase_price_9999_last_week_adjust',
 'purchase_price_sum_last_week_adjust',
 'role_last_login_duration',
 'role_lifespan',
 'role_login_last_now',
 'role_login_secondlast_now',
 'role_no',
 'purchase_price_sum_max_ratio',
 'purchase_price_sum_max_ratio_com2',
 'role_create_os_android',
 'is_role_transfer_to_new',
 'buy_rate_last_day_tech_unlock_trigger']



fea_l=''
for i in fea_list:
    fea_l+=i
    fea_l+=','

# client.reset()
# for i in trange(30):
#     ds = DS.add(days=-i).to_date_string()
#     temp = Template(open("pur_his.sql",'r').read())
#     [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]

client.reset()
for i in trange(30):
   
# i=9
    ds = DS.add(days=-i).to_date_string()
    temp = Template(open("com_feature.sql",'r').read())
    [client.execute_trino_query(q) for q in temp.render(feature_list=fea_l,ds = ds).split(';')]

# client.reset()
# for i in trange(60):
#     ds = DS.add(days=-i).to_date_string()
#     temp = Template(open("res_feature.sql",'r').read())
#     [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]


# client.reset()
# temp = Template(open("bundle.sql",'r').read())
# for i in trange(30):
#     ds = DS.add(days=-i).to_date_string()
#     [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:48<00:00,  1.61s/it]


In [ ]:
 
q = "select * from hive.ml_tmp_db.wgame_bunlde_dp_resfeature where dt = '{ds}' "
data = []
client.reset()
for i in trange(28):
    ds = DS.add(days=-i).to_date_string()
    client.execute_trino_query(q.format(ds = ds))
    data.append(client.to_pandas())
dataset = pd.concat(data, ignore_index=True)
dataset.to_csv("/home/jovyan/shared-fast/ruokaigu/datas/resfeature_data.csv")


In [ ]:
 
q = "select * from hive.ml_tmp_db.wgame_bunlde_price where dt = '{ds}' "
data = []
client.reset()
for i in trange(30):
    ds = DS.add(days=-i).to_date_string()
    client.execute_trino_query(q.format(ds = ds))
    data.append(client.to_pandas())
dataset = pd.concat(data, ignore_index=True)
dataset.to_csv("/home/jovyan/shared-fast/ruokaigu/datas/price_data.csv")

In [36]:
usd=pd.read_csv('/home/jovyan/ruokaigu/datas/resprice_data.csv')

In [43]:
usd['user_id'].sample(50000)

388604      2668392
1409120     4621024
2526980    23027299
2350231    18684573
984417      4440552
             ...   
1222688     4536943
1913586    11729481
3604661    33662698
950722      4406256
2649314    25532573
Name: user_id, Length: 50000, dtype: int64

In [68]:
pp=pd.read_csv('/home/jovyan/ruokaigu/datas/resprice_data.csv')
openlist=pp[pp['recharge']>0]['open_id']

In [36]:
openlist=pd.read_csv('/home/jovyan/ruokaigu/data_process/role_id.csv')

In [39]:
ol=openlist['0']

In [47]:
q = "select * from hive.ml_tmp_db.wgame_purhis where dt = '{ds}' and user_id in ({role_id_lis})"
# q = "select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis})and acc_id >'{acc_id} order by role_id,acc_id limit 1000000"
# q2="select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis}) order by role_id,acc_id  limit 1"

client.reset()

# print(role_ids_str)

# for i in trange(30):
#     ccc=0
#     ds = DS.add(days=-i).to_date_string()
    
#     for j in range(0,len(openlist),10000):
#         rlist=ol[j:j+10000]
#         role_ids_str = ','.join(f"'{role_id}'" for role_id in rlist)
        
#         # if ccc==0:
#         #     client.execute_trino_query(q2.format(ds = ds,role_id_lis=role_ids_str))
#         #     ccc=1
            
#         # else:
#         client.execute_trino_query(q.format(ds = ds,role_id_lis=role_ids_str))
        
#         data=client.to_pandas()
#         # last_openid=data.tail(1)[['acc_id']]
        
#         # if data.empty:
#         #     break
#         data.to_csv("/home/jovyan/ruokaigu/datas/purhis.csv",mode='a',index=False)

q = "select * from hive.ml_stage.wgame_bundle_content where dt = '{ds}' and user_id in ({role_id_lis})"
# q = "select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis})and acc_id >'{acc_id} order by role_id,acc_id limit 1000000"
# q2="select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis}) order by role_id,acc_id  limit 1"

client.reset()

# print(role_ids_str)

for i in trange(30):
    ccc=0
    ds = DS.add(days=-i).to_date_string()
    
    for j in range(0,len(openlist),10000):
        rlist=ol[j:j+10000]
        role_ids_str = ','.join(f"'{role_id}'" for role_id in rlist)
        
        # if ccc==0:
        #     client.execute_trino_query(q2.format(ds = ds,role_id_lis=role_ids_str))
        #     ccc=1
            
        # else:
        client.execute_trino_query(q.format(ds = ds,role_id_lis=role_ids_str))
        
        data=client.to_pandas()
        # last_openid=data.tail(1)[['acc_id']]
        
        # if data.empty:
        #     break
        data.to_csv("/home/jovyan/ruokaigu/datas/price.csv",mode='a',index=False)

100%|██████████| 30/30 [01:18<00:00,  2.61s/it]


In [48]:
q = "select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and role_id in ({role_id_lis})"
# q = "select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis})and acc_id >'{acc_id} order by role_id,acc_id limit 1000000"
# q2="select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis}) order by role_id,acc_id  limit 1"

client.reset()

# print(role_ids_str)

for i in trange(30):
    ccc=0
    ds = DS.add(days=-i).to_date_string()
    
    for j in range(0,len(openlist),10000):
        rlist=ol[j:j+10000]
        role_ids_str = ','.join(f"'{role_id}'" for role_id in rlist)
        
        # if ccc==0:
        #     client.execute_trino_query(q2.format(ds = ds,role_id_lis=role_ids_str))
        #     ccc=1
            
        # else:
        client.execute_trino_query(q.format(ds = ds,role_id_lis=role_ids_str))
        
        data=client.to_pandas()
        # last_openid=data.tail(1)[['acc_id']]
        
        # if data.empty:
        #     break
        data.to_csv("/home/jovyan/ruokaigu/datas/comfeac_.csv",mode='a',index=False)

100%|██████████| 30/30 [00:42<00:00,  1.43s/it]


In [85]:
q = "select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis})"
# q = "select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis})and acc_id >'{acc_id} order by role_id,acc_id limit 1000000"
# q2="select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id in ({role_id_lis}) order by role_id,acc_id  limit 1"

client.reset()

# print(role_ids_str)

for i in trange(30):
    ccc=0
    ds = DS.add(days=-i).to_date_string()
    
    for j in range(0,len(openlist),10000):
        rlist=ol[j:j+10000]
        role_ids_str = ','.join(f"'{role_id}'" for role_id in rlist)
        
        # if ccc==0:
        #     client.execute_trino_query(q2.format(ds = ds,role_id_lis=role_ids_str))
        #     ccc=1
            
        # else:
        client.execute_trino_query(q.format(ds = ds,role_id_lis=role_ids_str))
        
        data=client.to_pandas()
        # last_openid=data.tail(1)[['acc_id']]
        
        # if data.empty:
        #     break
        data.to_csv("/home/jovyan/ruokaigu/datas/comfeac_%s.csv"%ds,mode='a',index=False)
        # data.to_csv("/home/jovyan/ruokaigu/datas/comfea.csv",mode='a',index=False)


 67%|██████▋   | 2/3 [2:42:15<1:21:07, 4867.75s/it]


KeyboardInterrupt: 

In [ ]:
# q = "select distinct * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and (role_id, acc_id) > ('{role_id}', '{acc_id}') order by role_id,acc_id limit 1000000"
q = "select distinct * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and (role_id, acc_id) > ('{role_id}', '{acc_id}') order by role_id,acc_id limit 1000000"
q2="select * from hive.ml_stage.wgame_bundle_comfeature where dt = '{ds}' and acc_id NOT LIKE '%test%' order by role_id,acc_id  limit 1"
client.reset()
for i in trange(28):
    ccc=0
    while True:
        ds = DS.add(days=-i).to_date_string()
        if ccc==0:
            client.execute_trino_query(q2.format(ds = ds))
            ccc=1
            
        else:
            client.execute_trino_query(q.format(ds = ds,role_id=last_userid,acc_id=last_openid))
       
        data=client.to_pandas()
        # print(data.tail(1).columns)
        last_userid=data.tail(1)[['role_id']]
        last_openid=data.tail(1)['acc_id']
       
        if data.empty:
            break
        data.to_csv("/home/jovyan/ruokaigu/datas/comfea_%s.csv"%ds,mode='a',index=False)